In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE ROLE accountadmin;

-- create a development database for data science work
--CREATE OR REPLACE DATABASE HOL;

-- use the PUBLIC schema

use schema HOL.PUBLIC;


In [ ]:
-- create csv file format
CREATE OR REPLACE FILE FORMAT csv_ff
	TYPE=CSV
    SKIP_HEADER=1
    FIELD_DELIMITER=','
    TRIM_SPACE=TRUE
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
    REPLACE_INVALID_CHARACTERS=TRUE
    DATE_FORMAT=AUTO
    TIME_FORMAT=AUTO
    TIMESTAMP_FORMAT=AUTO; 

-- Create an external stage pointing to s3, to load sales data: 
CREATE OR REPLACE STAGE S3TASTYBYTES_Cortex 
COMMENT = 'Quickstart S3 Stage Connection'
url = 's3://sfquickstarts/frostbyte_tastybytes/mlpf_quickstart/'
file_format = csv_ff;



In [ ]:
-- Define Tasty Byte Sales Table
CREATE OR REPLACE table sales(
  	DATE TIMESTAMP_NTZ,
	PRIMARY_CITY VARCHAR,
	MENU_ITEM_NAME VARCHAR,
	TOTAL_SOLD NUMBER(38,0)
);

-- Ingest data from s3 into our table
COPY INTO sales 
FROM @S3TASTYBYTES_Cortex
FILES = ('mlpf_quickstart_vancouver_daily_sales.csv') 
FILE_FORMAT = csv_ff
ON_ERROR=CONTINUE ;

In [ ]:
-- Create Table containing the latest years worth of sales data: 
CREATE OR REPLACE view sales_forecast_input AS (
    SELECT
        to_timestamp_ntz(date) as timestamp,
        primary_city,
        menu_item_name,
        total_sold
    FROM
        sales 
    WHERE
        date > (SELECT max(date) - interval '1 year' FROM sales)
    GROUP BY
        all
);


In [ ]:
CREATE OR REPLACE VIEW lobster_sales AS (
    SELECT
        timestamp,
        total_sold
    FROM
        SALES_FORECAST_INPUT
    WHERE
        menu_item_name LIKE 'Lobster Mac & Cheese'
);


In [ ]:
CREATE OR REPLACE snowflake.ml.forecast lobstermac_forecast (
    INPUT_DATA => SYSTEM$REFERENCE('view', 'lobster_sales'),
    TIMESTAMP_COLNAME => 'timestamp',
    TARGET_COLNAME => 'total_sold'
);


In [ ]:
CREATE OR REPLACE TABLE lobstermac_predictions AS (
    SELECT *
    FROM TABLE( lobstermac_forecast!FORECAST(30) )
);


In [ ]:
CREATE OR REPLACE VIEW lobster_sales2 AS (
    SELECT TO_DATE(timestamp) AS date, total_sold, NULL as FORECAST, NULL as LOWER_BOUND, NULL AS UPPER_BOUND FROM lobster_sales
    UNION 
    SELECT TO_DATE(TS) AS date, NULL as total_sold, FORECAST, LOWER_BOUND, UPPER_BOUND from lobstermac_predictions
    ORDER BY date ASC
);


In [ ]:
df_test = session.table('lobster_sales2')
st.line_chart(df_test, x= 'DATE', y= 'TOTAL_SOLD')

In [ ]:
import plotly.express as px
st.plotly_chart(px.line(df_test, x="DATE" ,y=["TOTAL_SOLD","FORECAST"]), use_container_width=True)